# Textile Company
A textile company produces two types of fabric for the fashion industry, *plain* and *patterned* fabric.
The fabrics are woven from wool on a loom and are finished by hand labor.
A square meter of plain fabric requires $0.3$ Kg of wool, $6$ min of loom time, and $3$ hr of finishing
labor; a square meeter of patterned fabric requires $0.5$ Kg of wool, $5$ min of loom time, and $5$ hr of finishing labor.

For the next 3 months the company can procure up to 20 Kg of wool at the price of $\$$ 3/Kg, 5 hr of loom time at the price of $\$$ 3/minute, and 200 hr of labor at the price of $\$$ 10/hour. 
The company realizes a $\$$ 60/$m^2$ profit on the sale of plain fabric and a $\$$ 65/ $m^2$ profit on the sale of patterned fabric.
Assuming that all the fabric produced can be sold, determine how much of each type should the company produce in the next $3$ months, and how much inputs should be procured in order to maximize profits.


- Provide a mathematical formulation for the instance of the problem described in the text
- Find a solution using Gurobi

We start by providing a mathematical formulation

Let $x_a$ and $x_b$ denote the amount of plain and patterned fabric to produce, respectively, in $m^2$, $y_w$ the amount of wool (in Kg) to purchase, $y_l$ the amount of loom time (in minutes) to commit, and $y_h$ the amount of hand-finishing labor (in hours) to procure. The profit will be $60x_a+65x_b-3y_w-3y_l-10y_h$ and the amounts to be produced are limited by constraints on the amount of labor, wool and loom time. Thus, the mathematical model will be:

$$
\begin{align}
 \max ~& 60x_a+65x_b-3y_w-3y_l-10y_h \\
 s.t. ~& 0.3x_a+0.5x_b\leq y_w \\
  & 6x_a+5x_b\leq y_l \\
  & 3x_a+5x_b\leq y_h \\
  & y_w \leq 20 \\
  & y_l \leq 300 \\
  & y_h \leq 200 \\
  & x_a,x_b, y_w, y_l, y_h\geq 0
\end{align}
$$
The optimal solution has a profit of $\$$ $555$ and consists of procuring $15$ Kg of wool, $300$ minutes of loom time and $150$ hours of labor time, and producing $50$ square meters of only plain fabric.

To implement the model in Gurobi we import the necessary stuff

In [1]:
from gurobipy import Model,GRB

We create a Model object, which is a container for our mathematical model

In [2]:
m = Model('textile')

Restricted license - for non-production use only - expires 2023-10-25


We add the variables to the model

In [3]:
xa = m.addVar(0,GRB.INFINITY,vtype=GRB.CONTINUOUS,name="xa")
xb = m.addVar(0,GRB.INFINITY,vtype=GRB.CONTINUOUS,name="xb")
yw = m.addVar(0,GRB.INFINITY,vtype=GRB.CONTINUOUS,name="yw")
yl = m.addVar(0,GRB.INFINITY,vtype=GRB.CONTINUOUS,name="yl")
yh = m.addVar(0,GRB.INFINITY,vtype=GRB.CONTINUOUS,name="yh")

Then the objective

In [4]:
m.setObjective(60 * xa + 65 * xb - 3 * yw - 3 * yl - 10 * yh, sense= GRB.MAXIMIZE )

The constraints

In [5]:
constr1 = m.addConstr(0.3 * xa + 0.5 * xb - yw , sense= GRB.LESS_EQUAL, rhs=0)
m.addConstr(6 * xa + 5 * xb <= yl)
m.addConstr(3 * xa + 5 * xb <= yh)
m.addConstr(yw <= 20)
m.addConstr(yl <= 300)
m.addConstr(yh <= 200)

<gurobi.Constr *Awaiting Model Update*>

We solve the model

In [6]:
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 128 physical cores, 128 logical processors, using up to 32 threads
Optimize a model with 6 rows, 5 columns and 12 nonzeros
Model fingerprint: 0x8bca8799
Coefficient statistics:
  Matrix range     [3e-01, 6e+00]
  Objective range  [3e+00, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 3e+02]
Presolve removed 6 rows and 5 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5500000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.05 seconds (0.00 work units)
Optimal objective  5.550000000e+02


We get some useful information

In [7]:
print('Objective value: %g' % m.objVal)
print('%s %g' % (xa.varName, xa.x))
print('%s %g' % (xb.varName, xb.x))
print('%s %g' % (yw.varName, yw.x))
print('%s %g' % (yl.varName, yl.x))
print('%s %g' % (yh.varName, yh.x))

# We can obtain the same information using the model getAttr of the model object.
# In this case we need to pass the list of variables we are interested in
print("Variable names and values")
print(m.getAttr('varName', [yh]))
print(m.getAttr('x', [yh]))

print(m.getAttr('varName', [yh,yw]))
print(m.getAttr('x', [yh,yw]))

# Solution information
print("# Variables ", m.getAttr("NumVars"))
print("# Integer Variables ", m.getAttr("NumIntVars"))
print("# Constraints ", m.getAttr("NumConstrs"))

print("# Variables ", m.NumVars)
print("# Integer Variables ", m.NumIntVars)
print("# Binary Variables ", m.NumBinVars)
print("# Constraints ", m.NumConstrs)

# Duals
# Note that we need to assing the constraint to a variable, as we did with constr1
print('Dual = %g' % constr1.Pi)

Objective value: 555
xa 50
xb 0
yw 15
yl 300
yh 150
Variable names and values
['yh']
[150.0]
['yh', 'yw']
[150.0, 14.999999999999998]
# Variables  5
# Integer Variables  0
# Constraints  6
# Variables  5
# Integer Variables  0
# Binary Variables  0
# Constraints  6
Dual = 3
